In [1]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import random, gc, keras, os

from keras import backend as K
from keras.utils import plot_model
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.applications.vgg16 import VGG16

%matplotlib inline

Using TensorFlow backend.


# Define Parameters

In [2]:
img_size = (150, 150, 3)  # target image size
margin = 0.3              # triplet loss margin
batch_size = 64           # training batch size

# Define Triplet Network

In [60]:
# Define base network for triplet network
def base_net(input_shape=(150, 150, 3), trainable=False):
    """ define triplet network """
    # load pre-trained VGG16 model
    vgg16 = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
    vgg16.trainable = trainable
    
    # define sequential model
    model = Sequential(name='base_net')
    model.add(vgg16)
    model.add(Flatten(name='flatten'))
    model.add(Dense(512, activation='relu', name='fc1'))
    model.add(Dense(128, activation=None, name='fc2'))
    model.add(Lambda(lambda x: K.l2_normalize(x, axis=1), name='l2_norm'))
    
    return model

In [61]:
# Define triplet network
def triplet_net(base_model, input_shape=(150, 150, 3)):
    """ function to define triplet networks """
    # define input: anchor, positive, negative
    anchor = Input(shape=input_shape, name='anchor_input')
    positive = Input(shape=input_shape, name='positive_input')
    negative = Input(shape=input_shape, name='negative_input')
    
    # extract vector represent using CNN based model
    anchor_vec = base_model(anchor)
    pos_vec = base_model(positive)
    neg_vec = base_model(negative)
    
    # stack outputs
    stacks = Lambda(lambda x: K.stack(x, axis=1), name='output')([anchor_vec, pos_vec, neg_vec])

    # define inputs and outputs
    inputs=[anchor, positive, negative]
    outputs = stacks
    
    # define the triplet model
    model = Model(inputs=inputs, outputs=outputs, name='triplet_net')
    
    return model

In [62]:
# # Define triplet loss
# def triplet_loss(y_true, y_pred):
#     """ function to compute triplet loss
#         margin is predefined coded, manually change if needed
#     """
#     # define triplet margin
#     margin = 0.2
    
#     # get the prediction vector
#     anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
#     # compute distance
#     pos_distance = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis=-1)
#     neg_distance = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis=-1)
    
#     # compute loss
#     partial_loss = tf.subtract(pos_distance, neg_distance) + margin
#     full_loss = tf.reduce_sum(tf.maximum(partial_loss, 10.0))
    
#     return full_loss

In [83]:
# Define triplet loss
def triplet_loss(y_true, y_pred):
    """ function to compute triplet loss
        margin is predefined coded, manually change if needed
    """
    # define triplet margin
    margin = K.constant(0.5)
    zero = K.constant(0.0)
    
    # get the prediction vector
    anchor, positive, negative = y_pred[:, 0], y_pred[:, 1], y_pred[:, 2]
    
    # compute distance
    pos_distance = K.sum(K.square(anchor - positive), axis=1)
    neg_distance = K.sum(K.square(anchor - negative), axis=1)
    
    # compute loss
    partial_loss = pos_distance - neg_distance + margin
    full_loss = K.sum(K.maximum(partial_loss, zero), axis=0)
    
    return full_loss

# Test

In [84]:
# Define triplet network model
img_size = (224, 224, 3)  # target image size
base_model = base_net(input_shape=img_size, trainable=False)
triplet_model = triplet_net(base_model=base_model, input_shape=img_size)
triplet_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anchor_input (InputLayer)       (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
positive_input (InputLayer)     (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
negative_input (InputLayer)     (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
base_net (Sequential)           (None, 128)          27625920    anchor_input[0][0]               
                                                                 positive_input[0][0]             
          

In [85]:
# define optimizer
opt = keras.optimizers.Adam()

# compile the model
triplet_model.compile(optimizer=opt, loss=triplet_loss)

In [86]:
# zeros = np.zeros((2, 224, 224, 3))
# ones = np.ones((2, 224, 224, 3))
# inputs = np.concatenate((zeros, ones), axis=0)

inputs = np.zeros((5, 224, 224, 3))

In [87]:
base_pred = base_model.predict(inputs)
base_pred[:, :5]

array([[ 0.10081124, -0.01168546, -0.13378285, -0.00341781, -0.02500289],
       [ 0.10081124, -0.01168546, -0.13378285, -0.00341781, -0.02500289],
       [ 0.10081124, -0.01168546, -0.13378285, -0.00341781, -0.02500289],
       [ 0.10081124, -0.01168546, -0.13378285, -0.00341781, -0.02500289],
       [ 0.10081124, -0.01168546, -0.13378285, -0.00341781, -0.02500289]],
      dtype=float32)

In [89]:
triplet_pred = triplet_model.predict([inputs, inputs, inputs])
triplet_pred[:, 0][:, :5]

array([[ 0.10081124, -0.01168546, -0.13378285, -0.00341781, -0.02500289],
       [ 0.10081124, -0.01168546, -0.13378285, -0.00341781, -0.02500289],
       [ 0.10081124, -0.01168546, -0.13378285, -0.00341781, -0.02500289],
       [ 0.10081124, -0.01168546, -0.13378285, -0.00341781, -0.02500289],
       [ 0.10081124, -0.01168546, -0.13378285, -0.00341781, -0.02500289]],
      dtype=float32)

In [90]:
with tf.Session() as test:
    tf.set_random_seed(1)
    y_true = (None, None, None)
    y_pred = tf.zeros((5, 3, 128))
    loss = triplet_loss(y_true, y_pred)
    
    print("loss = " + str(loss.eval()))

loss = 2.5


In [92]:
tmp = np.ones((5, 3, 1))
triplet_model.test_on_batch([inputs, inputs, inputs], tmp)

2.5

In [93]:
triplet_model.evaluate([inputs, inputs, inputs], tmp)

5/5 [==============================] - 6s 1s/step


2.5

In [94]:
triplet_model.metrics_names

['loss']